In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.spatial.distance as dist

import iblofunmatch.inter as ibfm
output_dir = "output" 

import os 
os.makedirs(output_dir, exist_ok=True)
os.makedirs("plots/circles_plane", exist_ok=True)

In [ ]:
RandGen = np.random.default_rng(5)
C0 = ibfm.sampled_circle(7,7.5,50, RandGen)
C1 = ibfm.sampled_circle(0,2.5,50, RandGen)
X = np.vstack((C0, C1))
y= [0]*C0.shape[0] + [1] * C1.shape[0]
fig, ax = plt.subplots(ncols=1, figsize=(5,5))
ax.scatter(C0[:,0], C0[:,1], color=mpl.colormaps["RdBu"](0.1), s=30)
ax.scatter(C1[:,0], C1[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=50, marker="x")
ax.set_axis_off()
plt.savefig("plots/circles_plane/circle_inside_blob.png")

In [ ]:
exp_ibfm = []
for cidx in range(2):
    class_indices = np.nonzero(np.array(y)==cidx)[0].tolist()
    exp_ibfm.append(ibfm.get_IBloFunMatch_subset(None, X, class_indices, output_dir, num_it=4, max_rad=-1, points=True))
# end for over classes 
indices_block_0 = [] 
indices_block_1 = [] 
for cidx in range(2):
    indices_block_0 += exp_ibfm[cidx]["block_function_0"]
    indices_block_1 += exp_ibfm[cidx]["block_function_1"]

not_indices_block_0 = [i for i in range(exp_ibfm[0]["X_barcode_0"].shape[0]) if i not in indices_block_0]
not_indices_block_1 = [i for i in range(exp_ibfm[0]["X_barcode_1"].shape[0]) if i not in indices_block_1]

In [ ]:
block_0_0 = exp_ibfm[0]["block_function_0"]
block_0_1 = exp_ibfm[1]["block_function_0"]
[i for i in block_0_0 if i in block_0_1]

In [ ]:
max_rad_1 = np.max([[np.max(ibfm_out["S_barcode_1"]), np.max(ibfm_out["X_barcode_1"])] for ibfm_out in exp_ibfm])*1.2
fig, ax = plt.subplots(nrows=2, ncols=4, figsize=(12,6))
for cidx, ibfm_out in enumerate(exp_ibfm):
    ibfm.plot_matching(ibfm_out, ax[cidx, [0,1]], fig, block_function=True, dim=0, codomain_int=not_indices_block_0)
    ibfm.plot_matching(ibfm_out, ax[cidx, [2,3]], fig, max_rad=max_rad_1, block_function=True, codomain_int=not_indices_block_1)
# end for 
plt.savefig("plots/circles_plane/circ_in_blob_block.png")

## Example combining two classes containing a cycle each, and both are nested

Change the radious of the smaller cycle along a range.

In [ ]:
RandGen = np.random.default_rng(5)
C0 = ibfm.sampled_circle(3,3.5,50, RandGen)
rad_list = [0.5,1,1.5,2,2.5]
C1_list = []
X_list = []
y_list = []
for rad in rad_list:
    C1_list.append(ibfm.sampled_circle(rad*0.8,rad,50, RandGen))
    X_list.append(np.vstack((C0, C1_list[-1])))
    y_list.append([0]*C0.shape[0] + [1] * C1_list[-1].shape[0])
# end for 
fig, ax = plt.subplots(ncols=len(rad_list), figsize=(5*len(rad_list),5))
for i, C1 in enumerate(C1_list):
    ax[i].scatter(C0[:,0], C0[:,1], color=mpl.colormaps["RdBu"](0.1), s=30)
    ax[i].scatter(C1[:,0], C1[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=50, marker="x")
    ax[i].set_axis_off()

In [ ]:
all_exp_ibfm = [] 
indices_block_0_all = []
indices_block_1_all = []
for X, y in zip(X_list, y_list):
    exp_ibfm = []
    for cidx in range(2):
        class_indices = np.nonzero(np.array(y)==cidx)[0].tolist()
        exp_ibfm.append(ibfm.get_IBloFunMatch_subset(None, X, class_indices, output_dir, num_it=4, max_rad=-1, points=True))
    # end for over classes 
    all_exp_ibfm.append(exp_ibfm)
    indices_block_0 = []
    indices_block_1 = []
    for cidx in range(2):
        indices_block_0 += exp_ibfm[cidx]["block_function_0"]
        indices_block_1 += exp_ibfm[cidx]["block_function_1"]

    
    not_indices_block_0 = [i for i in range(exp_ibfm[0]["X_barcode_0"].shape[0]) if i not in indices_block_0]
    not_indices_block_1 = [i for i in range(exp_ibfm[0]["X_barcode_1"].shape[0]) if i not in indices_block_1]
    indices_block_0_all.append(not_indices_block_0)
    indices_block_1_all.append(not_indices_block_1)

In [ ]:
max_rad_0 = np.max([[np.max(ibfm_out["S_barcode_0"]), np.max(ibfm_out["X_barcode_0"])] for ibfm_out in exp_ibfm])*1.2
fig, ax = plt.subplots(nrows=len(all_exp_ibfm), ncols=5, figsize=(15,3*len(all_exp_ibfm)))
for idx_exp, exp_ibfm in enumerate(all_exp_ibfm):
    not_indices_block_0 = indices_block_0_all[idx_exp]
    #compute repeated blocks
    block_0_0 = exp_ibfm[0]["block_function_0"]
    block_0_1 = exp_ibfm[1]["block_function_0"]
    repeated_block_0 = [i for i in block_0_0 if i in block_0_1]
    # plot matchings
    ibfm.plot_matching(exp_ibfm[0], ax[idx_exp, [0,1]], fig, max_rad=max_rad_0, block_function=True, codomain_int=not_indices_block_0, dim=0, repeated_codomain=repeated_block_0)
    ibfm.plot_matching(exp_ibfm[1], ax[idx_exp, [2,3]], fig, max_rad=max_rad_0, block_function=True, codomain_int=not_indices_block_0, dim=0, repeated_codomain=repeated_block_0)
    # plot points 
    C1 = C1_list[idx_exp]
    ax[idx_exp, 4].scatter(C0[:,0], C0[:,1], color=mpl.colormaps["RdBu"](0.1), s=30)
    ax[idx_exp, 4].scatter(C1[:,0], C1[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=50, marker="x")
    ax[idx_exp, 4].set_axis_off()
plt.savefig("plots/circles_plane/concentric_evolution_dim_0.png")

In [ ]:
max_rad_1 = np.max([[np.max(ibfm_out["S_barcode_1"]), np.max(ibfm_out["X_barcode_1"])] for ibfm_out in exp_ibfm])*1.2
fig, ax = plt.subplots(nrows=len(all_exp_ibfm), ncols=5, figsize=(15,3*len(all_exp_ibfm)))
for idx_exp, exp_ibfm in enumerate(all_exp_ibfm):
    not_indices_block_1 = indices_block_1_all[idx_exp]
    ibfm.plot_matching(exp_ibfm[0], ax[idx_exp, [0,1]], fig, max_rad=max_rad_1, block_function=True, codomain_int=not_indices_block_1)
    ibfm.plot_matching(exp_ibfm[1], ax[idx_exp, [2,3]], fig, max_rad=max_rad_1, block_function=True, codomain_int=not_indices_block_1)
    # plot points 
    C1 = C1_list[idx_exp]
    ax[idx_exp, 4].scatter(C0[:,0], C0[:,1], color=mpl.colormaps["RdBu"](0.1), s=30)
    ax[idx_exp, 4].scatter(C1[:,0], C1[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=50, marker="x")
    ax[idx_exp, 4].set_axis_off()
plt.savefig("plots/circles_plane/concentric_evolution.png")

## Example combining two classes containing a cycle and additional points each. 

This example illustrates something that is detected using the block function but that it is not necessarily detected using the induced partial matching.

In [ ]:
# old two circles and two blobs
# RandGen = np.random.default_rng(5)
# C0 = ibfm.sampled_circle(3,3.5,50, RandGen)
# ball_00 = ibfm.sampled_circle(0,1,3, RandGen)
# ball_01 = ibfm.sampled_circle(0,1,3, RandGen)+[7,0]
# C0 = np.vstack((C0, ball_00, ball_01))
# C1 = ibfm.sampled_circle(2,2.5,30, RandGen)
# ball_1 = ibfm.sampled_circle(0,2.5,20, RandGen)+[7,0]
# C1 = np.vstack((C1, ball_1))
# X = np.vstack((C0, C1))
# y= [0]*C0.shape[0] + [1] * C1.shape[0]

In [ ]:
RandGen = np.random.default_rng(5)
C0 = ibfm.sampled_circle(3,3.5,50, RandGen)
C1 = ibfm.sampled_circle(1.2,1.5,25, RandGen)+[10,0]
ball_1 = ibfm.sampled_circle(0,2.5,25, RandGen)
C1 = np.vstack((C1, ball_1))
X = np.vstack((C0, C1))
y= [0]*C0.shape[0] + [1] * C1.shape[0]

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(10,5))
ax.scatter(C0[:,0], C0[:,1], color=mpl.colormaps["RdBu"](0.1), s=30)
ax.scatter(C1[:,0], C1[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=50, marker="x")
ax.set_axis_off()

In [ ]:
exp_ibfm = []
for cidx in range(2):
    class_indices = np.nonzero(np.array(y)==cidx)[0].tolist()
    exp_ibfm.append(ibfm.get_IBloFunMatch_subset(None, X, class_indices, output_dir, num_it=4, max_rad=-1, points=True))
# end for over classes 
indices_block_0 = [] 
indices_block_1 = [] 
for cidx in range(2):
    indices_block_0 += exp_ibfm[cidx]["block_function_0"]
    indices_block_1 += exp_ibfm[cidx]["block_function_1"]

not_indices_block_0 = [i for i in range(exp_ibfm[0]["X_barcode_0"].shape[0]) if i not in indices_block_0]
not_indices_block_1 = [i for i in range(exp_ibfm[0]["X_barcode_1"].shape[0]) if i not in indices_block_1]

In [ ]:
max_rad_1 = np.max([[np.max(ibfm_out["S_barcode_1"]), np.max(ibfm_out["X_barcode_1"])] for ibfm_out in exp_ibfm])*1.2
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(12,6))
for cidx, ibfm_out in enumerate(exp_ibfm):
    ibfm.plot_blofun_0_diag(ibfm_out, ax[cidx, 0], draw_hist=False)
    ibfm.plot_matching(ibfm_out, ax[cidx, [1,2]], fig, max_rad=max_rad_1, block_function=True, codomain_int=not_indices_block_1)
# end for 

Changing this same example by just a little change, we obtain a similar output, but with a different matching.

In [ ]:
# old
# RandGen = np.random.default_rng(5)
# C0 = ibfm.sampled_circle(3,3.5,40, RandGen)
# ball_00 = ibfm.sampled_circle(0,1,5, RandGen)
# ball_01 = ibfm.sampled_circle(0,1,5, RandGen)+[7,0]
# C0 = np.vstack((C0, ball_00, ball_01))
# C1 = ibfm.sampled_circle(2,2.5,30, RandGen)+[7,0]
# ball_1 = ibfm.sampled_circle(0,2.5,20, RandGen)
# C1 = np.vstack((C1, ball_1))
# X = np.vstack((C0, C1))
# y= [0]*C0.shape[0] + [1] * C1.shape[0]

In [ ]:
RandGen = np.random.default_rng(5)
C0 = ibfm.sampled_circle(3,3.5,50, RandGen)
C1 = ibfm.sampled_circle(1.2,1.5,25, RandGen)
ball_1 = ibfm.sampled_circle(0,2.5,25, RandGen)+[10,0]
C1 = np.vstack((C1, ball_1))
X = np.vstack((C0, C1))
y= [0]*C0.shape[0] + [1] * C1.shape[0]
fig, ax = plt.subplots(ncols=1, figsize=(10,5))
ax.scatter(C0[:,0], C0[:,1], color=mpl.colormaps["RdBu"](0.1), s=30)
ax.scatter(C1[:,0], C1[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=50, marker="x")
ax.set_axis_off()

In [ ]:

exp_ibfm = []
for cidx in range(2):
    class_indices = np.nonzero(np.array(y)==cidx)[0].tolist()
    exp_ibfm.append(ibfm.get_IBloFunMatch_subset(None, X, class_indices, output_dir, num_it=4, max_rad=-1, points=True))
# end for over classes 
indices_block_0 = [] 
indices_block_1 = [] 
for cidx in range(2):
    indices_block_0 += exp_ibfm[cidx]["block_function_0"]
    indices_block_1 += exp_ibfm[cidx]["block_function_1"]

not_indices_block_0 = [i for i in range(exp_ibfm[0]["X_barcode_0"].shape[0]) if i not in indices_block_0]
not_indices_block_1 = [i for i in range(exp_ibfm[0]["X_barcode_1"].shape[0]) if i not in indices_block_1]
max_rad_1 = np.max([[np.max(ibfm_out["S_barcode_1"]), np.max(ibfm_out["X_barcode_1"])] for ibfm_out in exp_ibfm])*1.2
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(12,6))
for cidx, ibfm_out in enumerate(exp_ibfm):
    ibfm.plot_blofun_0_diag(ibfm_out, ax[cidx, 0], draw_hist=False)
    ibfm.plot_matching(ibfm_out, ax[cidx, [1,2]], fig, max_rad=max_rad_1, block_function=True, codomain_int=not_indices_block_1)
# end for 